In [1]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

from datetime import datetime
from datetime import date

import base64
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

import gymnasium as gym
from gymnasium import logger as gymlogger
# from gym.wrappers import Monitor # deprecated 2023 - https://stackoverflow.com/questions/71520568/importerror-cannot-import-name-monitor-from-gym-wrappers
from gymnasium.wrappers.record_video import RecordVideo
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

print("\n",date.today(), datetime.now().strftime("%H:%M:%S"),"GMT") # timestamp is greenwich time
print("OK.")


 2023-03-10 10:36:06 GMT
OK.


In [2]:

# A simple neural network object
# source: https://github.com/AsmaBRZ/Evolutionary-algorithms/blob/master/fixed_structure_nn_numpy.py

# coding: utf-8

import numpy as np

## Suppress TF info messages

import os

def sigmoid(x):
    return 1./(1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)


def gen_simplemlp(n_in, n_out, n_hidden_layers=2, n_neurons_per_hidden=5):
    n_neurons = [n_neurons_per_hidden]*n_hidden_layers if np.isscalar(n_neurons_per_hidden) else n_neurons_per_hidden
    i = Input(shape=(n_in,))
    x = i
    for n in n_neurons:
        x = Dense(n, activation='sigmoid')(x)
    o = Dense(n_out, activation='tanh')(x)
    m = Model(inputs=i, outputs=o)
    return m


class SimpleNeuralControllerNumpy():
    def __init__(self, n_in, n_out, n_hidden_layers=2, n_neurons_per_hidden=5, params=None):
        self.dim_in = n_in
        self.dim_out = n_out
        # if params is provided, we look for the number of hidden layers and neuron per layer into that parameter (a dicttionary)
        if (not params==None):
            if ("n_hidden_layers" in params.keys()):
                n_hidden_layers=params["n_hidden_layers"]
            if ("n_neurons_per_hidden" in params.keys()):
                n_neurons_per_hidden=params["n_neurons_per_hidden"]
        self.n_per_hidden = n_neurons_per_hidden
        self.n_hidden_layers = n_hidden_layers
        self.weights = None
        self.n_weights = None
        self.init_random_params()
        self.out = np.zeros(n_out)
        #print("Creating a simple mlp with %d inputs, %d outputs, %d hidden layers and %d neurons per layer"%(n_in, n_out,n_hidden_layers, n_neurons_per_hidden))


    def init_random_params(self):
        if(self.n_hidden_layers > 0):
            self.weights = [np.random.random((self.dim_in,self.n_per_hidden))] # In -> first hidden
            self.bias = [np.random.random(self.n_per_hidden)] # In -> first hidden
            for i in range(self.n_hidden_layers-1): # Hidden -> hidden
                self.weights.append(np.random.random((self.n_per_hidden,self.n_per_hidden)))
                self.bias.append(np.random.random(self.n_per_hidden))
            self.weights.append(np.random.random((self.n_per_hidden,self.dim_out))) # -> last hidden -> out
            self.bias.append(np.random.random(self.dim_out))
        else:
            self.weights = [np.random.random((self.dim_in,self.dim_out))] # Single-layer perceptron
            self.bias = [np.random.random(self.dim_out)]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])

    def get_parameters(self):
        """
        Returns all network parameters as a single array
        """
        flat_weights = np.hstack([arr.flatten() for arr in (self.weights+self.bias)])
        return flat_weights

    def set_parameters(self, flat_parameters):
        """
        Set all network parameters from a single array
        """
        i = 0 # index
        to_set = []
        self.weights = list()
        self.bias = list()
        if(self.n_hidden_layers > 0):
            # In -> first hidden
            w0 = np.array(flat_parameters[i:(i+self.dim_in*self.n_per_hidden)])
            self.weights.append(w0.reshape(self.dim_in,self.n_per_hidden))
            i += self.dim_in*self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                w = np.array(flat_parameters[i:(i+self.n_per_hidden*self.n_per_hidden)])
                self.weights.append(w.reshape((self.n_per_hidden,self.n_per_hidden)))
                i += self.n_per_hidden*self.n_per_hidden
            # -> last hidden -> out
            wN = np.array(flat_parameters[i:(i+self.n_per_hidden*self.dim_out)])
            self.weights.append(wN.reshape((self.n_per_hidden,self.dim_out)))
            i += self.n_per_hidden*self.dim_out
            # Samefor bias now
            # In -> first hidden
            b0 = np.array(flat_parameters[i:(i+self.n_per_hidden)])
            self.bias.append(b0)
            i += self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                b = np.array(flat_parameters[i:(i+self.n_per_hidden)])
                self.bias.append(b)
                i += self.n_per_hidden
            # -> last hidden -> out
            bN = np.array(flat_parameters[i:(i+self.dim_out)])
            self.bias.append(bN)
            i += self.dim_out
        else:
            n_w = self.dim_in*self.dim_out
            w = np.array(flat_parameters[:n_w])
            self.weights = [w.reshape((self.dim_in,self.dim_out))]
            self.bias = [np.array(flat_parameters[n_w:])]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])

    def predict(self,x):
        """
        Propagate
        """
        if(self.n_hidden_layers > 0):
            #Input
            a = np.matmul(x,self.weights[0]) + self.bias[0]
            y = sigmoid(a)
            # hidden -> hidden
            for i in range(1,self.n_hidden_layers-1):
                a = np.matmul(y, self.weights[i]) + self.bias[i]
                y = sigmoid(a)
            # Out
            a = np.matmul(y, self.weights[-1]) + self.bias[-1]
            out = tanh(a)
            return out
        else: # Simple monolayer perceptron
            return tanh(np.matmul(x,self.weights[0]) + self.bias[0])

    def __call__(self,x):
        """Calling the controller calls predict"""
        return self.predict(x)

print("\n",date.today(), datetime.now().strftime("%H:%M:%S"),"GMT") # timestamp is greenwich time
print("OK.")


 2023-03-10 10:36:06 GMT
OK.


In [3]:

env = gym.make('CartPole-v1', render_mode='human')
# env = wrap_env( gym.make('CartPole-v1') ) # colab-specific # needs py-game (import in earlier cell)

observation, info = env.reset()

#for _ in range(1000):
#while True:
maxEvaluations = 10
evaluation = 0
while evaluation < maxEvaluations:
    env.render()
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
      evaluation = evaluation + 1
      observation, info = env.reset()
      print ("obs:",observation,type(observation),observation.shape)
      #show_video(False) # colab-specific

env.close()

# show_video() # colab-specific

obs: [-0.01718654  0.00780112  0.02278244 -0.02092541] <class 'numpy.ndarray'> (4,)
obs: [-0.03872223  0.01280517  0.01505941  0.04271357] <class 'numpy.ndarray'> (4,)
obs: [-0.02709858 -0.0141231   0.04538532  0.04090934] <class 'numpy.ndarray'> (4,)
obs: [-0.01625654  0.00461658 -0.01663846  0.00144274] <class 'numpy.ndarray'> (4,)
obs: [-0.02753758  0.02384411  0.02081367  0.04301481] <class 'numpy.ndarray'> (4,)
obs: [-0.01335328  0.02161715 -0.01359328 -0.02682093] <class 'numpy.ndarray'> (4,)
obs: [ 0.0095457  -0.01175741  0.02942055 -0.04101792] <class 'numpy.ndarray'> (4,)
obs: [ 0.01807426  0.02142726  0.04410785 -0.0292872 ] <class 'numpy.ndarray'> (4,)
obs: [-0.0453118  -0.01616802  0.01359317  0.0102906 ] <class 'numpy.ndarray'> (4,)
obs: [ 0.04835679  0.04047988 -0.02707644  0.03287368] <class 'numpy.ndarray'> (4,)


# Question B-1

En partant de l'exemple de la documentation et du code fourni pour le réseau de neurones dans la première cellule ci-dessous, faites une expérience pour optimiser les paramètres d'un réseau de neurones contrôlant un pendule inversé (environnement 'CartPole-v1') avec CMA-ES. Imposez une limite à 500 pas de temps au-delà de laquelle l'expérience est considérée comme réussie.

使用文档中的例子和为下面第一个单元格中的神经网络提供的代码，执行一个实验来优化用CMA-ES控制倒立摆的神经网络的参数（CartPole-v1环境）。设置500个时间步数的限制，超过这个时间步数就认为实验成功。

In [4]:
# Exemple de création d'un réseau de neurones multi-couches ("multi-layered perceptron")

nbInputs = 4
nbOutputs = 1
nbHiddenLayers = 2
nbNeuronsPerLayer = 5

nn=SimpleNeuralControllerNumpy(nbInputs,nbOutputs,nbHiddenLayers,nbNeuronsPerLayer)

# NN parameters

theta = nn.get_parameters() # get default parameters
for i in range(len(theta)):
  theta[i] = random.random()*2.0-1.0 # set our own parameter values
nn.set_parameters(theta)
print ("NN parameters:",theta)

# running NN

observation = np.array( [ -0.03760372,0.00724161,-0.03663132,0.01225319 ], np.float64 )

print ("Input values:", observation)

outputValues = nn.predict(observation)

print ("Output values:", outputValues)


NN parameters: [-0.21863377 -0.25891941  0.27295175  0.828277    0.50751095  0.80650617
  0.52375932 -0.32483662 -0.68591724  0.6684494  -0.2342329   0.06805185
 -0.18531291  0.0604336  -0.29983903 -0.89446921  0.44398634 -0.43973455
 -0.068092    0.08385179  0.61662309  0.51756617 -0.3742084   0.0668632
 -0.34095977  0.62863114  0.08673652  0.52568422 -0.22516899 -0.51382779
  0.58550383 -0.41254502 -0.0758776  -0.38717772  0.57303667  0.89517319
 -0.78107305 -0.28601248  0.46629837  0.67344502  0.15252293 -0.84970598
  0.91824432 -0.46018826  0.18838473 -0.60418321 -0.3733826  -0.65015385
 -0.01326887  0.7676656  -0.42479103  0.52414    -0.93094318  0.73436807
 -0.17486381  0.78960326  0.52941792 -0.17165798 -0.88015825  0.57305472
  0.40667645]
Input values: [-0.03760372  0.00724161 -0.03663132  0.01225319]
Output values: [0.08865271]


In [13]:
from cma import CMAEvolutionStrategy

def evaluate_nn(nn, theta):
    observation, info = env.reset()
    nn.set_parameters(theta)
    done = False
    total_reward = 0
    num_steps = 0
    while not done and total_reward < 500:
        output = nn.predict(observation)[0]
        action = 0 if output < 0 else 1
        observation, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        total_reward += reward
        num_steps += 1
    env.close()
    return -total_reward

env = gym.make('CartPole-v1')
observation, info = env.reset()
nbInputs = env.observation_space.shape[0]
nbOutputs = env.action_space.n
nbHiddenLayers = 2
nbNeuronsPerLayer = 5
nn=SimpleNeuralControllerNumpy(nbInputs,nbOutputs,nbHiddenLayers,nbNeuronsPerLayer)
initial_params = nn.get_parameters()
# optimize the parameters using CMA-ES
es = CMAEvolutionStrategy(initial_params, 0.5)
while not es.stop() and es.result.fbest > -500:
    solutions = es.ask()
    fitness_list = [evaluate_nn(nn,s) for s in solutions ]
    es.tell(solutions, fitness_list)
print('best fitness:', es.best.f)
print('mean fitness:', np.mean(fitness_list))
print('best NN parameters:', es.best.x)

(8_w,16)-aCMA-ES (mu_w=4.8,w_1=32%) in dimension 67 (seed=573436, Fri Mar 10 10:45:54 2023)
best fitness: -500.0
mean fitness: -61.8125
best NN parameters: [ 1.59475564e+00  5.01047694e-01 -5.44594455e-01 -8.13165205e-01
  6.05432100e-01 -1.82124551e+00  3.36233642e-01  1.04225215e+00
  6.02841885e-01  5.59786900e-01  1.13420781e+00  2.15721695e+00
  5.92929208e-01 -7.46770380e-01  2.23375332e+00 -7.55442791e-01
  1.03071673e+00  3.84907672e-01  5.85019835e-01  1.32164473e+00
  6.73201193e-01  6.80194115e-01 -3.08729286e-01  4.87985930e-02
  1.97301620e+00  6.72388710e-01  6.74057786e-01  2.32499642e+00
  9.94895985e-01 -1.20435512e+00 -1.47086287e+00  1.32965317e-01
  9.84186371e-01  5.08847503e-01 -1.40786473e-01  9.28702301e-01
  5.08347095e-01  1.38661532e+00  4.04127942e-01  2.26145875e+00
 -3.75085574e-01 -3.87366610e-01 -4.60073786e-01 -7.57253525e-01
  4.97909886e-01 -4.68723733e-01  6.86423531e-01  3.16643223e-02
  1.20710254e+00 -1.01283317e-01  3.32714456e-01 -6.75936191e-01